In [ ]:
!pip install kaggle
!pip install opendatasets
!pip install pandas

In [ ]:
import opendatasets as od
 
od.download("https://www.kaggle.com/datasets/stackoverflow/stacksample/")


In [ ]:
import pandas as pd
from io import StringIO

df = pd.read_csv('stacksample/Answers.csv', encoding = "ISO-8859-1")

df.head()

In [ ]:
df = pd.read_csv('stacksample/Tags.csv', encoding = "ISO-8859-1")

df.head()

In [ ]:
import pandas as pd
from io import StringIO

df = pd.read_csv('stacksample/Questions.csv', encoding = "ISO-8859-1")

df.head()